In [2]:
import os

print(os.name)

import platform

print(platform.system())

print(platform.release())

print(os.uname())

posix
Linux
3.10.0-1160.88.1.el7.x86_64
posix.uname_result(sysname='Linux', nodename='coka.fe.infn.it', release='3.10.0-1160.88.1.el7.x86_64', version='#1 SMP Tue Mar 7 15:41:52 UTC 2023', machine='x86_64')


In [4]:
import healpy as hp
for i in range(17):
    try:
        print(i,hp.nside2npix(i))
    except:
        print("could not compute it")
    finally:
        pass

0 0
1 12
2 48
3 108
4 192
5 300
6 432
7 588
8 768
9 972
10 1200
11 1452
12 1728
13 2028
14 2352
15 2700
16 3072
